In [15]:
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# Membaca data dari file CSV
# Try reading the file with different encodings
encodings_to_try = ['utf-8', 'latin1', 'ISO-8859-1']

for encoding in encodings_to_try:
    try:
        df = pd.read_csv('Daftar_Skripsi.csv', encoding=encoding)
        print(f"File successfully read using encoding: {encoding}")
        # Further processing or analysis on the dataframe
        break  # Stop the loop if the file is read successfully
    except UnicodeDecodeError:
        print(f"Could not read the file with encoding: {encoding}")

# If none of the encodings work, consider further inspection or conversion of the file

Could not read the file with encoding: utf-8
File successfully read using encoding: latin1


Preprocessing

In [16]:
# Fungsi untuk membersihkan simbol dan angka dari teks
def remove_symbols_and_numbers(text):
    # Menggunakan ekspresi reguler untuk menghilangkan simbol dan angka
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    return cleaned_text
    
# Terapkan remove_symbols_and_numbers pada kolom 'JudulSkripsi' dan membuat kolom baru judul_cleaned
df['judul_cleaned'] = df['JudulSkripsi'].apply(remove_symbols_and_numbers)

Tokenisasi dan Case Folding

In [17]:
# Tokenisasi dan case folding pada kolom judul
df['judul_tokenized'] = df['judul_cleaned'].apply(lambda x: word_tokenize(str(x)))
df['judul_lower'] = df['judul_tokenized'].apply(lambda x: [word.lower() for word in x])


Filtering

In [23]:
# stopword from file
list_stopwords = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

In [25]:
# Function to remove stopwords
def stopwords_removal(words, stopwords):
    return [word for word in words if word.lower() not in stopwords]

# Read stopwords from the file stopwords.txt
with open('stopwords.txt', 'r') as file:
    stopwords = file.read().splitlines()

# Assuming 'data' contains the DataFrame and 'judul_lower' column is the list of words
# Apply stopwords_removal to each list in 'judul_lower' column
df['judul_no_stopwords'] = df['judul_lower'].apply(lambda x: stopwords_removal(x, stopwords))


In [26]:
pip install Sastrawi

You should consider upgrading via the 'c:\Users\frans\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [31]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Function to perform stemming
def apply_stemming(words):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)  # Join stemmed words back into a sentence

# Apply stemming to each list in 'judul_lower' column
df['judul_stemmed'] = df['judul_no_stopwords'].apply(apply_stemming)

Kumpulan kata dasar = df['judul_stemmed']

In [36]:
# Assuming 'data' contains the DataFrame and 'judul_stemmed' column has the stemmed text

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the stemmed text data using TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(df['judul_stemmed'])

# Convert TF-IDF matrix to DataFrame for further analysis
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names(), index=df.index)

In [37]:
from sklearn.cluster import KMeans

# Initialize KMeans clustering
num_clusters = 5  # Ubah jumlah kluster sesuai kebutuhan
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Fit KMeans to the TF-IDF matrix
kmeans.fit(tfidf_matrix)

# Predict cluster labels for the data points
df['cluster_label'] = kmeans.labels_

# Optional: Print the cluster centers
print("Cluster Centers:")
print(kmeans.cluster_centers_)  # Centroids of clusters

# Optional: Analyze the cluster assignments
cluster_counts = df['cluster_label'].value_counts()
print("\nCluster Counts:")
print(cluster_counts)  # Number of data points in each cluster

Cluster Centers:
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.02473705 0.        ]
 [0.         0.         0.02122812 ... 0.         0.         0.02538005]
 [0.0164457  0.01333895 0.00978795 ... 0.01279429 0.         0.        ]]

Cluster Counts:
4    27
2    14
3    12
1     8
0     5
Name: cluster_label, dtype: int64


In [46]:
# Misalkan 'df' adalah DataFrame yang Anda miliki
df['cluster_label'] = df['cluster_label'] + 1

# Print DataFrame setelah menambahkan 1 pada setiap nilai di kolom 'cluster_label'
print(df[['JudulSkripsi', 'cluster_label']])


                                         JudulSkripsi  cluster_label
0   Alat Pendeteksi Kebocoran Gas Lpg Berbasis Ard...              1
1   Sistem Pendukung Keputusan Penilaian Kinerja P...              4
2   Membangun Pengenalan Hewan Augmented Reality B...              2
3   Alat Penyiram Tanaman Otomatis Menggunakan Ard...              1
4   Desain Dan Implementasi E-Learning Pada Lkp Ac...              5
..                                                ...            ...
61  Aplikasi Pengolahan Data Sparepart Pada Pt Kal...              5
62  Aplikasi Monitoring Bukti Potong Pph Pasal 15 ...              5
63       Aplikasi Kebugaran Tubuh Berbasis Multimedia              5
64  Sistem Informasi Data Kepegawaian Pada Kantor ...              5
65  Aplikasi Pengolahan Data Service Handphone Pad...              5

[66 rows x 2 columns]


In [47]:
# Misalkan 'df' adalah DataFrame yang Anda miliki
sorted_df = df[['JudulSkripsi', 'cluster_label']].sort_values('cluster_label')

# Print DataFrame yang sudah diurutkan berdasarkan kolom 'cluster_label'
print(sorted_df)

                                         JudulSkripsi  cluster_label
0   Alat Pendeteksi Kebocoran Gas Lpg Berbasis Ard...              1
43  Penerapan Spk Decission Tree Menggunakan Algor...              1
37  Sistem Pendeteksi Peringatan Dini Keamanan Rum...              1
23  Game Edukasi Drag And Drop Budaya Nusantara Un...              1
3   Alat Penyiram Tanaman Otomatis Menggunakan Ard...              1
..                                                ...            ...
34  Sistem Informasi Pengelolaan Media Pembelajara...              5
40  Aplikasi Pemilihan Handphone Pada Parindo Cell...              5
41  Implementasi Absensi Siswa Berbasis Sms Gatewa...              5
45  Pengembangan Sistem Pengolahan Data Gaji Karya...              5
65  Aplikasi Pengolahan Data Service Handphone Pad...              5

[66 rows x 2 columns]


In [48]:
print(df.dtypes)


id                     int64
JudulSkripsi          object
NamaPeneliti          object
Tahun                  int64
ProgramStudi          object
judul_cleaned         object
judul_tokenized       object
judul_lower           object
judul_no_stopwords    object
judul_stemmed         object
cluster_label          int32
dtype: object
